# 4. Google Pagerank Algorithm (10 points)

**Keywords**: Pagerank, Power Method

**About the dataset**: \
*DBpedia* (from "DB" for "database") is a project aiming to extract structured content from the information created in the Wikipedia project. This structured information is made available on the World Wide Web. DBpedia allows users to semantically query relationships and properties of Wikipedia resources, including links to other related datasets. for more info, see: https://en.wikipedia.org/wiki/DBpedia. \
We will download two files from the data respository:
* The first file -- **redirects_en.nt.bz2** -- contains redirects link for a page. Let A redirect to B and B redirect to C. Then we will replace article A by article C wherever needed.
* The second file -- **page_links_en.nt.bz2** -- contains pagelinks which are links within an article to other wiki article.

Note that the data is both files is a list of lines which can be split into 4 parts:
* The link to first article.
* Whether it is a redirect, or a pagelink.
* The link to second article.
* A fullstop.

#### <font color="red">Note:</font> Any line which cannot be split into 4 parts is skipped from consideration.

**Agenda**:
* In this programming challenge, you will be implementing the [*google pagerank algorithm*](https://towardsdatascience.com/pagerank-algorithm-fully-explained-dc794184b4af) to determine the most important articles.
* This will be done by computing the principal eigenvector of the article-article graph adjacency matrix.
* In this challenge, you will be applying the *power method* to extract the principal eigenvector from the adjacency matrix. 
* Using the computed eigenvector, we can assign each article a eigenvector-centrality score. Then we can determine the most important articles.

**Environment**:
Ensure following libraries are installed
- sklearn
- numpy

Also ensure that you have around **4 GB** of memory.

**Note:**
* Run all the cells in order.
* **Do not edit** the cells marked with !!DO NOT EDIT!!
* Only **add your code** to cells marked with !!!! YOUR CODE HERE !!!!
* Do not change variable names, and use the names which are suggested.



---



In [3]:
# !! DO NOT EDIT !!
# imports
import pickle
from bz2 import BZ2File
import bz2
import os
from datetime import datetime
import pprint
from time import time
import numpy as np
from urllib.request import urlopen
import scipy.sparse as sparse
pp = pprint.PrettyPrinter(indent=4)

In [4]:
# !! DO NOT EDIT !!
# download the dataset and store files in local

# dbpedia download urls
redirects_url = "http://downloads.dbpedia.org/3.5.1/en/redirects_en.nt.bz2"
page_links_url = "http://downloads.dbpedia.org/3.5.1/en/page_links_en.nt.bz2"

# extarct the file-names from the urls. Needed to load the files later
redirects_filename = redirects_url.rsplit("/", 1)[1] # redirects_en.nt.bz2 ~ 59MB
page_links_filename = page_links_url.rsplit("/", 1)[1] # page_links_en.nt.bz2 ~ 850MB

resources = [
    (redirects_url, redirects_filename),
    (page_links_url, page_links_filename),
]

# download the files
# this will take some time
for url, filename in resources:
    if not os.path.exists(filename):
        print("Downloading data from '%s', please wait..." % url)
        opener = urlopen(url)
        open(filename, "wb").write(opener.read())
        print()

In [5]:
# !! DO NOT EDIT !!
# load the data from the downloaded files
# this may take some time

#read redirects file
redirects_file = bz2.open(redirects_filename, mode='rt')
redirects_data = redirects_file.readlines()
redirects_file.close()

# pagelinks data has 119M entries
# We will only consider the first 5M for this question
pagelinks_file = bz2.open(page_links_filename, mode='rt')
pagelinks_data = [next(pagelinks_file) for x in range(5000000)] 
pagelinks_file.close()

In [6]:
# !! DO NOT EDIT !!
# look at the size of the data and some examples
print ('The number of entries in redirects:', len(redirects_data))
print ('A couple of examples from redirects:')
print (redirects_data[10000:10002])
print ('\n')

print ('The number of entries in pagelinks:', len(pagelinks_data))
print ('A couple of examples from pagelinks:')
print (pagelinks_data[100000:100002])

The number of entries in redirects: 4082533
A couple of examples from redirects:
['<http://dbpedia.org/resource/Proper_superset> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/Subset> .\n', '<http://dbpedia.org/resource/Jean_Paul_Sartre> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/Jean-Paul_Sartre> .\n']


The number of entries in pagelinks: 5000000
A couple of examples from pagelinks:
['<http://dbpedia.org/resource/Antipope> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Council_of_Constance> .\n', '<http://dbpedia.org/resource/Antipope> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Pope_Alexander_V> .\n']


#### <font color="red">Note:</font> It is worth noting here that each article is uniquely represented by its URL, or rather, the last segment of its URL



---



### **(a)** Define a function `get_article_name` which takes as input the URL string, and extracts the last segment of the URL, which we can call as article name. (1 point)

In [7]:
#######
# !!!! YOUR CODE HERE !!!!
len_of_prefix = len("http://dbpedia.org/resource/")
last_segment_slice = slice(len_of_prefix + 1, -1)

def get_article_name(url):
  return url[last_segment_slice]

#######

In [8]:
# !! DO NOT EDIT !!
# some unit tests to validate your solution
assert get_article_name('<http://dbpedia.org/resource/Pope_Alexander_V>') == 'Pope_Alexander_V'
assert get_article_name('<http://dbpedia.org/resource/Jean-Paul_Sartre>') == 'Jean-Paul_Sartre'



---


### **(b)** Define a function `resolve_redirects` which takes as input a list of redirect lines, and returns a map between the initial and the resolved redirect page. (2 points)

e.g.: input = \
[
'\<http://dbpedia.org/resource/A> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/B> .\n',\
'\<http://dbpedia.org/resource/B> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/C> .\n',\
'\<http://dbpedia.org/resource/C> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/D> .\n',\
'\<http://dbpedia.org/resource/X> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/Z> .\n'
]

output = {'A': 'D', 'B': 'D', 'C': 'D', 'X': 'Z'}

#### <font color="red">Note:</font> Remember to ignore malformed lines which are those which do not split in 4 parts.

In [9]:
#######
# !!!! YOUR CODE HERE !!!!
def resolve_redirects(redirects_url):
  output = {}

  for url in redirects_url:
    # split the redirect url into 3 parts
    components = url.split(' ') 

    # ignoring malformed lines
    if len(components) != 4:
      continue

    # the starting redirect is the first element of components list
    start = get_article_name(components[0])

    #the ending redirect is the second to last element of components list
    end = get_article_name(components[-2])

    # if a line redirects to itself, we ignore it
    if start == end:
      continue

    # add start end to dictionary
    if start not in output:
      output[start] = end

  print("Updating immediate redirects to final redirects")
  for i, start in enumerate(output.keys()):
    final_redirect = None
    immediate_redirect = output[start]
    alreadySeen = {start}
    
    while True:
        final_redirect = immediate_redirect
        immediate_redirect = output.get(immediate_redirect)
        if immediate_redirect is None or immediate_redirect in alreadySeen:
            break
        alreadySeen.add(immediate_redirect)

    output[start] = final_redirect

    # printing checkpoint
    if i % 1000000 == 0:
        print("Completed Line: ", i)
    
  return output

#######

In [10]:
# !! DO NOT EDIT !!
# some unit tests to validate your solution
test_input = ['<http://dbpedia.org/resource/A> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/B> .\n', 
              '<http://dbpedia.org/resource/B> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/C> .\n', 
              '<http://dbpedia.org/resource/C> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/D> .\n', 
              '<http://dbpedia.org/resource/X> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/Z> .\n']

test_output = {'A': 'D', 'B': 'D', 'C': 'D', 'X': 'Z'}

assert resolve_redirects(test_input) == test_output

Updating immediate redirects to final redirects
Completed Line:  0




---



### **(c)** Create article-article adjacency matrix. 
### Let the number of articles $n$. The adjacency matrix should have a value $A[i][j]=1$ if there is a link from $i$ to $j$. Note that the matrix may not be symmetric. This matrix would have rows as source, and columns as destinations. However, for further sections, we need it the other way round. Therefore, return $A^\top$ matrix. 
### Create a function `make_adjacency_matrix` that takes as input the resolved redirect map from part (b), and the list from `pagelinks_data`.
### Return a tuple of `(index_map, A')`, where `index_map` is a map of each article to a unique number between $0$ and $n-1$, also its unique numerical id. `A` is the adjacency matrix in [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) format. `A'` is the transpose of matrix `A`. (2 points)


#### <font color="red">Note:</font> Take care that if A redirects to D and X redirects to Y, and there is a pagelink entry from A to X, then the resolved pagelink entry should be D to Y.

In [11]:
#######
# !!!! YOUR CODE HERE !!!!

def get_index(resolved_redirects, index_map, n):
    # Find and return unique integer index of article names after resolving redirects 
    n = resolved_redirects.get(n, n)
    return index_map.setdefault(n, len(index_map))


def make_adjacency_matrix(resolved_redirects, page_links_data):
    
    # Computing the Index Map
    print("Computing the Index Map")
    index_map = dict()
    list_of_links = list()
    
    for k, line in enumerate(page_links_data):
        segment = line.split()
        if len(segment) != 4:
            continue
        i = get_index(resolved_redirects, index_map, get_article_name(segment[0]))
        j = get_index(resolved_redirects, index_map, get_article_name(segment[2]))
        list_of_links.append((i, j))
        
        # checkpoint
        if k % 1000000 == 0:
            print("Completed Line:", k)

    # Computing the Adjacency Matrix
    print("Computing the Adjacency Matrix")
    X = sparse.lil_matrix((len(index_map), len(index_map)), dtype=np.float32)

    for i, j in list_of_links:
        X[i, j] = 1.0
    del list_of_links

    # Converting to CSR representation
    X = X.tocsr()
    # Taking transpose of Adjacency Matrix   
    X = X.transpose()

    # returning Index Map and Adjacency Matrix  
    return index_map, X

#######

In [12]:
# !! DO NOT EDIT !!
# some unit tests to validate your solution

test_redirects = {'A': 'D', 'B': 'D', 'C': 'D', 'X': 'Z', 'K':'L', 'M':'N'}
test_pagelinks_data = ['<http://dbpedia.org/resource/A> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/X> .\n', '<http://dbpedia.org/resource/L> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/N> .\n', '<http://dbpedia.org/resource/P> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Q> .\n']

test_output_index_map = {'D': 0, 'Z': 1, 'L': 2, 'N': 3, 'P': 4, 'Q': 5}
test_output_adjacency_matrix = np.array([[0., 1., 0., 0., 0., 0.],
                                           [0., 0., 0., 0., 0., 0.],
                                           [0., 0., 0., 1., 0., 0.],
                                           [0., 0., 0., 0., 0., 0.],
                                           [0., 0., 0., 0., 0., 1.],
                                           [0., 0., 0., 0., 0., 0.]])

output_index_map, output_A = make_adjacency_matrix(test_redirects, test_pagelinks_data) 

assert output_index_map == test_output_index_map
np.testing.assert_array_equal(output_A.toarray(), test_output_adjacency_matrix.T)

Computing the Index Map
Completed Line: 0
Computing the Adjacency Matrix




---



### **(d)** Apply the above functions on the dataset to create adjacency matrix $A$ and other relevant maps as directed below. Then apply `SVD` from sklearn on the adjacency matrix to determine principal singular vectors. (2 points)

In [13]:
#######
# !!!! YOUR CODE HERE !!!!
# 1. with redirects_data as input, use the resolve_redirects function to generate the redirects_map
# redirects_map = ________
redirects_map = resolve_redirects(redirects_data)

# 2. with redirects map from previous step pagelinks_data as inputs, use the make_adjacency_matrix to generate index_map and adjacency_matrix
# index_map, X = ________
index_map, X = make_adjacency_matrix(redirects_map, pagelinks_data)

# 3. using index_map, create a reverse_index_map, which has the article name as key, and its index as value
# reverse_index_map = ________
reverse_index_map = {i: article_name for article_name, i in index_map.items()}
#######

Updating immediate redirects to final redirects
Completed Line:  0
Completed Line:  1000000
Completed Line:  2000000
Completed Line:  3000000
Completed Line:  4000000
Computing the Index Map
Completed Line: 0
Completed Line: 1000000
Completed Line: 2000000
Completed Line: 3000000
Completed Line: 4000000
Computing the Adjacency Matrix


In [14]:
# !! DO NOT EDIT !!
# Now we will save the csr matrix, index_map and reverse_index_map in pickle files
# so that we do not have to recompute steps (a)-(d) next time we load the program
# (Note: beneficial only when working on local machine, as colab session times out)
PATH='./'
pickle.dump(X, open(PATH+'X.pkl', 'wb'))
pickle.dump(index_map, open(PATH+'index_map.pkl', 'wb'))
pickle.dump(reverse_index_map, open(PATH+'reverse_index_map.pkl', 'wb'))

# free up RAM
del(redirects_data, pagelinks_data)

! ---------- Checkpoint ----------- !

In [15]:
# !! DO NOT EDIT !!
# Load the data from here
PATH='./'
X = pickle.load(open(PATH+'X.pkl', 'rb'))
index_map = pickle.load(open(PATH+'index_map.pkl', 'rb'))
reverse_index_map =  pickle.load(open(PATH+'reverse_index_map.pkl', 'rb'))

Apply ```randomized_svd``` from sklearn on the adjacency matrix. Extract top 5 components and run for 3 iterations.

In [16]:
#######
# !!!! YOUR CODE HERE !!!!
# U, s, V = __________
from sklearn.decomposition import randomized_svd
U, s, V = randomized_svd(X, 5, n_iter=3)

#######

In [17]:
# !! DO NOT EDIT !!
# now, we print the names of the wikipedia related strongest components of the
# principal singular vector which should be similar to the highest eigenvector
print("Top wikipedia pages according to principal singular vectors")
pp.pprint([reverse_index_map[i] for i in np.abs(U.T[0]).argsort()[-10:]])
pp.pprint([reverse_index_map[i] for i in np.abs(V[0]).argsort()[-10:]])

Top wikipedia pages according to principal singular vectors
[   'England',
    'Spain',
    'Italy',
    'Canada',
    'Japan',
    'Germany',
    'World_War_II',
    'France',
    'United_Kingdom',
    'United_States']
['1989', '1971', '1975', '1970', '2006', '1972', '1996', '1966', '1967', '2007']




---



### **(e)** The pagerank algorithm 
### In this final section we will implementing the google pagerank algorithm by computing principal eigenvector using power iteration method. (3 points)

### To start with the power iteration method, we first need to make the matrix $X$ obtained in (d) *column stochastic*. A column stochastic matrix is a matrix in which each element represents a probability and the sum each column adds up to 1. Recall that $X$ is a matrix where the rows represent the destination and columns represents the source. The probability of visiting any destination from the source $s$ is $1/k$, where $k$ is the total number of outgoing links from $s$. Use this information to make the matrix column stochastic.  


In [28]:
#######
# !!!! YOUR CODE HERE !!!!
# Make a copy of X
Y = X.copy()

# get 1D flattened array total outgoing links corresponding to each index
outgoing_links = np.asarray(Y.sum(axis=1)).ravel()

print("Making the matrix column stochastic")

# Looping over nonzero indices
for i in outgoing_links.nonzero()[0]:
    # update value of probability by dividing it by corresponding total outgoing links
    Y.data[Y.indptr[i] : Y.indptr[i + 1]] *= 1.0 / outgoing_links[i]
    
#######

Making the matrix column stochastic


### *Dangling Nodes*: There may exisit some pages which have no outgoing links. These are called as dangling nodes. If a random surfer just follows outgoing page links, then such a person can never leave a dangling node. We cannot just skip such a node, as there may be many pages pointing to this page, and could therefore be important. 
### To solve this problem, we introduce **teleportation** which says that a random surfer will visit an outgoing link with $\beta$ probability and can randomly jump to some other page with a $(1-\beta)/n$ probability (like through bookmarks, directly going through URL, etc.). Here $n$ is the total number of pages under consideration, and $\beta$ is called the damping factor. So now, the modified transition matrix is:
### $R = \beta X + \frac{(1-\beta)}{n} I_{n\times n}$

### where $X$ is the column stochastic matrix from previous step, and $I_{n\times n}$ is a $n\times n$ identity matrix.

### Using the transition matrix $R$, use the power iteration method to solve for the principal eigenvector $\mathbf{p}_{n\times 1}$. Start with an initial guess of $\mathbf{p}_{n\times 1}=[\frac{1}{n}, \frac{1}{n}, ..., \frac{1}{n}]$, which intuitively represents that a random surfer can start at any page with a $\frac{1}{n}$ probability. Use a damping factor of $0.85$, and perform a maximum of 100 iterations.

### Report the top 10 page names which correspond to the top 10 scores (magnitudes) in the principal eigenvector.




In [29]:
#######
# !!!! YOUR CODE HERE !!!!
def power_method(X, beta=0.85, max_iterations=100):
    
    n = X.shape[0]
    tolerance = 1e-10

    #initial guess, every value = 1/n 
    scores = np.array([(1.0 / n) for i in range(n)], dtype=np.float32)

    oldEigenValue = 0

    for i in range(max_iterations):
        prev_scores = scores
        # modifying scores using new transition matrix
        scores = beta*prev_scores*X + (1.0 - beta) * prev_scores.sum() / n
        
        # eigenvalue calculation
        newEigenValue = np.abs(scores).max()

        if newEigenValue == 0.0:
            newEigenValue = 1.0
        
        # normalizing scores
        scores = scores/newEigenValue 

        # error calculation
        error = np.abs(newEigenValue - oldEigenValue)
        if error < tolerance:
            return scores
        
        oldEigenValue = newEigenValue

    return scores


# Calculating principal eigenvector scores using power method
scores = power_method(Y, max_iterations=100)

# Reporting the top 10 page names which correspond to the top 10 scores (magnitudes) in the principal eigenvector.
print("Top 10 page names")
pp.pprint([reverse_index_map[i] for i in np.abs(scores).argsort()[-10:]])

#######

Top 10 page names
[   'Telecommunications_in_Brazil',
    'Politics_of_Romania',
    'List_of_Star_Trek:_The_Next_Generation_episodes',
    'Foreign_relations_of_Afghanistan',
    'Demographics_of_Poland',
    'Foreign_relations_of_Syria',
    'Foreign_relations_of_South_Africa',
    'List_of_fictional_robots_and_androids',
    'Foreign_relations_of_Uruguay',
    'Foreign_relations_of_Turkey']




---

